In [1]:
!pip install tensorflow matplotlib numpy scikit-learn --user -U

  Using cached tensorflow-2.13.1-cp38-cp38-win_amd64.whl (1.9 kB)
  Using cached matplotlib-3.7.5-cp38-cp38-win_amd64.whl (7.5 MB)
  Using cached scikit_learn-1.3.2-cp38-cp38-win_amd64.whl (9.3 MB)
  Using cached contourpy-1.1.1-cp38-cp38-win_amd64.whl (477 kB)
  Using cached importlib_resources-6.4.5-py3-none-any.whl (36 kB)
  Using cached fonttools-4.57.0-cp38-cp38-win_amd64.whl (1.5 MB)
  Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
  Using cached numpy-1.22.4-cp38-cp38-win_amd64.whl (14.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

print(tf.__version__) # verifica veloce versione TF


ImportError: cannot import name 'rcParams' from 'matplotlib.axes' (C:\Users\net.PC1-18\AppData\Roaming\Python\Python38\site-packages\matplotlib\axes\__init__.py)

In [ ]:
# Carica CIFAR-10: (X_train, y_train), (X_test, y_test)
(X_train_full, y_train_full), (X_test_full, y_test_full) = cifar10.load_data()
y_train_full = y_train_full.flatten()
y_test_full = y_test_full.flatten()

# CIFAR-10 label map ufficiale
label_names = ['airplane', 'automobile', 'bird','cat','deer' ,'dog','frog','horse', 'ship', 'truck']
cat_label, dog_label = 3, 5

# Filtra solo gatti e cani
train_mask = (y_train_full == cat_label) | (y_train_full == dog_label)
test_mask = (y_test_full == cat_label) | (y_test_full == dog_label)

X_train_cd = X_train_full[train_mask]
y_train_cd = y_train_full[train_mask]
X_test_cd = X_test_full[test_mask]
y_test_cd = y_test_full[test_mask]

# Rimappa etichette: cat->0, dog->1
y_train_cd = (y_train_cd == dog_label).astype(np.int32)
y_test_cd = (y_test_cd == dog_label).astype(np.int32)

X_train_cd.shape, X_test_cd.shape, np.bincount(y_train_cd), np.bincount(y_test_cd)

In [ ]:
def show_examples(x, y, class_names=('cat', 'dog'), n=12):
    plt.figure(figsize=(10, 4))
    idx = np.random.choice(len(x), size=n, replace=False)
    for i, j in enumerate(idx):
        plt.subplot(2, n//2, i+1)
        plt.imshow(x[j])
        plt.title(class_names[y[j]])
        plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
# Opzione: riduci il numero di campioni per velocizzare il training in aula
# Imposta a None per usare tutti i dati disponibili
SAMPLES_PER_CLASS_TRAIN = 3000 # es: 3000 cat + 3000 dog = 6000
SAMPLES_PER_CLASS_TEST = 500

def subsample_balanced(X, y, samples_per_class) :
    if samples_per_class is None:
        return X, y
    X_out, y_out = [], []
    for cls in [0,1]:
        idx = np.where(y == cls) [0]
        idx = np.random.choice(idx, size=min(samples_per_class, len(idx) ), replace=False)
        X_out.append(X[idx])
        y_out.append(y[idx] )
    return np.concatenate(X_out), np.concatenate(y_out)

X_train_small, y_train_small = subsample_balanced(X_train_cd, y_train_cd, SAMPLES_PER_CLASS_TRAIN)
X_test_small, y_test_small = subsample_balanced(X_test_cd, y_test_cd, SAMPLES_PER_CLASS_TEST)

# Mescola il training
perm = np. random. permutation(len(X_train_small) )
X_train_small, y_train_small = X_train_small[perm], y_train_small[perm]

# Normalizzazione [0,1]
X_train_small = X_train_small.astype('float32') / 255.0
X_test_small = X_test_small.astype('float32') / 255.0

X_train_small.shape, X_test_small.shape

In [ ]:
def build_cnn(input_shape=(32,32,3)):
    model = models. Sequential( [
        layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=input_shape),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(128, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D((2,2)),
        layers. Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid') # output binario
    ])
    model. compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = build_cnn()
model. summary ()


In [ ]:
EPOCHS = 4
history = model.fit(
    X_train_small, y_train_small,
    validation_split=0.1,
    epochs=EPOCHS,
    batch_size=64,
    verbose=1
)

In [ ]:
# Valutazione su test
y_prob = model.predict(X_test_small). ravel()
y_pred = (y_prob >= 0.5).astype(int)

acc = accuracy_score(y_test_small, y_pred)
cm = confusion_matrix(y_test_small, y_pred)

print(f"Accuracy test: {acc :. 3f}")
print("Confusion Matrix (rows=veri, cols=predetti):\n", cm)
print("\nReport:\n", classification_report(y_test_small, y_pred, target_names=['cat','dog'] ) )